#### **Task 10**: Do bigger delays lead to bigger fuel comsumption per passenger? 
We need to do four things to answer this as accurate as possible:
- Find out average monthly delay per air carrier (monthly delay is sum of all delays in 1 month)
- Find out distance covered monthly by different air carriers
- Find out number of passengers that were carried by different air carriers
- Find out total fuel comsumption per air carrier.

Use this information to get the average fuel comsumption per passenger per km. Is this higher for the airlines with bigger average delays?

In [44]:
## The marketing airline is the airline that sold the ticket, and the operating carrier is the carrier that actually carried the passenger

# https://en.wikipedia.org/wiki/Codeshare_agreement

In [1]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
pd.set_option('display.max_rows', None)

In [2]:
df_flights_pass = pd.read_csv(r'..\..\2_data\Flights_Passenger_Combined_Sample.csv', sep=',')
df_flights_pass.head()

,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,air_time,distance,fl_date,mkt_unique_carrier,...,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source
0,10821,BWI,"Baltimore, MD",12889,LAS,"Las Vegas, NV",267,2106,2018-09-19,WN,...,US,United States,6,652,1,2017,5,5,L,DU
1,14100,PHL,"Philadelphia, PA",13487,MSP,"Minneapolis, MN",139,980,2019-05-14,DL,...,US,United States,6,699,1,2015,12,2,F,DU
2,11292,DEN,"Denver, CO",14831,SJC,"San Jose, CA",129,948,2019-08-09,UA,...,US,United States,6,614,1,2018,8,2,F,DU
3,10397,ATL,"Atlanta, GA",11298,DFW,"Dallas/Fort Worth, TX",122,731,2019-01-25,DL,...,US,United States,6,678,1,2018,1,2,F,DU
4,13487,MSP,"Minneapolis, MN",13930,ORD,"Chicago, IL",54,334,2018-04-20,DL,...,US,United States,6,614,1,2019,2,1,F,DU


In [4]:
df_flights_pass.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 72 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   origin_airport_id      100000 non-null  int64  
 1   origin                 100000 non-null  object 
 2   origin_city_name       100000 non-null  object 
 3   dest_airport_id        100000 non-null  int64  
 4   dest                   100000 non-null  object 
 5   dest_city_name         100000 non-null  object 
 6   air_time               100000 non-null  int64  
 7   distance               100000 non-null  int64  
 8   fl_date                100000 non-null  object 
 9   mkt_unique_carrier     100000 non-null  object 
 10  branded_code_share     100000 non-null  object 
 11  mkt_carrier            100000 non-null  object 
 12  mkt_carrier_fl_num     100000 non-null  int64  
 13  op_unique_carrier      100000 non-null  object 
 14  tail_num               100000 non-nul

In [3]:
df_fuel = pd.read_csv(r'..\..\2_data\fuel_consumption.csv', sep=',')
df_fuel.head()

,month,airline_id,unique_carrier,carrier,carrier_name,carrier_group_new,sdomt_gallons,satl_gallons,spac_gallons,slat_gallons,...,sdomt_cost,satl_cost,spac_cost,slat_cost,sint_cost,ts_cost,tdomt_cost,tint_cost,total_cost,year
0,1,NaN,NaN,0JQ,NaN,1,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0,5100,0.0,5100,2016
1,1,21352.0,0WQ,0WQ,Avjet Corporation,1,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0,396216,140239.0,536455,2016
2,1,21645.0,23Q,23Q,Songbird Airways Inc.,1,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0,0,0.0,0,2016
3,1,21652.0,27Q,27Q,"Jet Aviation Flight Services, Inc.",1,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0,0,0.0,0,2016
4,1,20408.0,5V,5V,Tatonduk Outfitters Limited d/b/a Everts Air A...,1,260848.0,0.0,0.0,0.0,...,522405,0.0,0.0,0.0,0.0,522405,569497,0.0,569497,2016


Step 1 - Find out average monthly delay per air carrier (monthly delay is sum of all delays in 1 month)


In [7]:
df_flights_pass[['dep_delay','arr_delay','carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']].sum().head(10) 

dep_delay              940067.0
arr_delay              336375.0
carrier_delay          353983.0
weather_delay           60794.0
nas_delay              239225.0
security_delay           1441.0
late_aircraft_delay    480155.0
dtype: float64

In [5]:
# get sum of delay per month per carrier

filter = df_flights_pass[['month','op_unique_carrier','dep_delay']] 
carrier_delay_monthly = filter.groupby(['op_unique_carrier','month']).sum('dep_delay')
carrier_delay_monthly.head(20)

dep_delay
op_unique_carrier month           
9E                1         2048.0
                  2         2452.0
                  3         1663.0
                  4         1895.0
                  5         1884.0
                  6         3269.0
                  7         2868.0
                  8         2926.0
                  9         3086.0
                  10        2441.0
                  11        2116.0
                  12        1971.0
AA                1         7042.0
                  2         9035.0
                  3         8513.0
                  4        11629.0
                  5         9223.0
                  6         9566.0
                  7        12314.0
                  8         9505.0

In [28]:
# create month column

df_flights['month'] = pd.DatetimeIndex(df_flights['fl_date']).month  

In [29]:
# get delay per month per carrier

filter = df_flights[['month','op_unique_carrier','dep_delay']] 
carrier_delay_monthly = filter.groupby(['op_unique_carrier','month']).sum('dep_delay')
carrier_delay_monthly.head(20)

dep_delay
op_unique_carrier month           
9E                1         1966.0
                  2         3446.0
                  3         1548.0
                  4         2389.0
                  5         1418.0
                  6         6088.0
                  7         6704.0
                  8         6625.0
                  9         1278.0
                  10        1792.0
                  11         351.0
                  12        5700.0
9K                3          -75.0
                  5         -100.0
AA                1         7399.0
                  2        10614.0
                  3        13192.0
                  4        10293.0
                  5        12256.0
                  6        16746.0

In [37]:
# get average monthly delay per carrier

avg_monthly_delay = carrier_delay_monthly.groupby('op_unique_carrier').mean('dep_delay').reset_index()
avg_monthly_delay

,op_unique_carrier,distance
0,9E,1.404173e+05
1,9K,1.354500e+03
2,AA,1.067937e+06
3,AS,2.917309e+05
4,AX,3.358417e+04
5,B6,3.937174e+05
6,C5,1.890067e+04
7,CP,5.044058e+04
8,DL,9.617262e+05
9,EM,1.716000e+03


Step 2 - Find out distance covered monthly by different air carriers

In [32]:
# get distance per month per carrier

filter = df_flights[['month','op_unique_carrier','distance']] 
carrier_delay_monthly = filter.groupby(['op_unique_carrier','month']).sum('distance')
carrier_delay_monthly.head(20)

distance
op_unique_carrier month          
9E                1        158454
                  2        120542
                  3        130213
                  4        116440
                  5         99519
                  6        149324
                  7        110829
                  8        210361
                  9        180432
                  10       139898
                  11       109724
                  12       159272
9K                3          1290
                  5          1419
AA                1       1111331
                  2        982814
                  3       1203523
                  4       1075037
                  5        856212
                  6       1152391

In [36]:
# get average monthly distance per carrier

avg_monthly_distance = carrier_delay_monthly.groupby('op_unique_carrier').mean('distance').reset_index()
avg_monthly_distance

,op_unique_carrier,distance
0,9E,1.404173e+05
1,9K,1.354500e+03
2,AA,1.067937e+06
3,AS,2.917309e+05
4,AX,3.358417e+04
5,B6,3.937174e+05
6,C5,1.890067e+04
7,CP,5.044058e+04
8,DL,9.617262e+05
9,EM,1.716000e+03


In [47]:
fuc = df_flights['op_unique_carrier'].unique()
len(fuc)

27

In [62]:
muc = df_flights['mkt_unique_carrier'].unique()
len(muc)

10

In [63]:
codeshare = df_flights['branded_code_share'].unique()
len(codeshare)

15

In [60]:
puc = df_p['unique_carrier'].unique()
len(puc)

329

Step 3 - Find out number of passengers that were carried by different air carriers

DL     10667
UA      8856
WN      8335
AA      7065
OO      6405
FX      5105
EV      3432
H6      3240
G4      2788
YX      2681
AS      2600
5X      2524
NK      2232
MQ      2227
F9      2187
OH      2078
B6      2001
9E      1950
GV      1863
YV      1486
8E      1372
SY      1299
7S      1230
G7      1005
09Q      835
ZW       797
5Y       751
AX       678
04Q      659
GL       638
27Q      616
CP       543
U7       504
8V       502
PT       502
Y4       492
KE       490
C5       470
S5       467
AC       452
WS       432
5V       427
K2       420
J5       417
EM       410
V8       390
LH       385
2O       371
QX       371
ABX      362
AMQ      357
13Q      334
0WQ      321
LF       318
US       313
4Y       306
KAQ      271
HA       268
BA       265
M6       260
DY       255
KAH      247
4O       246
XP       241
9K       236
AM       229
KO       223
7H       222
NC       222
QK       212
3F       208
K3       206
4W       206
GFQ      206
3M       203
VX       190
CA       180

In [41]:
# get passengers per month per carrier

filter = df_p[['month','unique_carrier','passengers']]
carrier_passengers_monthly = filter.groupby(['unique_carrier','month']).sum('passengers')
carrier_passengers_monthly.head(20)

passengers
unique_carrier month            
04Q            1             308
               2             255
               3             617
               4             323
               5             438
               6            2301
               7            3118
               8            1899
               9            2149
               10            260
               11           1356
               12            517
07Q            2              30
               3               0
09Q            1            8677
               2           14349
               3           21437
               4            6064
               5           12880
               6            2610